# Unsupervised Learning - Topic Modeling

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\julie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train_data = pd.read_pickle("datasets/all_train_data_embedding.pkl")
train_data

## LDA

### Small cleaning

In [6]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = train_data.avis_en.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['best', 'insurance', 'price', 'solutions', 'listening', 'speed', 'recommend', 'company', 'attractive', 'prices', 'quality', 'services', 'speed']


### Build LDA model

In [7]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=20, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2)]
[(0, '0.107*"vehicle" + 0.092*"car" + 0.062*"euros" + 0.055*"month" + 0.054*"accident" + 0.036*"also" + 0.036*"responsible" + 0.034*"reimbursed" + 0.033*"guarantees" + 0.031*"offered"'), (1, '0.070*"contract" + 0.040*"time" + 0.037*"year" + 0.037*"since" + 0.029*"even" + 0.027*"make" + 0.026*"one" + 0.025*"call" + 0.024*"made" + 0.022*"care"'), (2, '0.299*"insurer" + 0.250*"disaster" + 0.078*"great" + 0.044*"changed" + 0.042*"incompetence" + 0.031*"door" + 0.028*"came" + 0.023*"mechanic" + 0.021*"consideration" + 0.013*"agents"'), (3, '0.123*"phone" + 0.092*"advisor" + 0.076*"subscription" + 0.073*"contact" + 0.052*"response" + 0.045*"impossible" + 0.038*"management" + 0.037*"contributions" + 0.033*"professional" + 0.027*"answers"'), (4, '0.136*"asked" + 0.069*"subscribe" + 0.065*"signed" + 0.062*"top" + 0.053*"start" + 0.046*"future" + 0.042*"options" + 0.039*"unacceptable" + 0.031*"gest

### Get the most important topics for each document

In [8]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

C:\Users\julie\AppData\Local\Temp\ipykernel_16564\1893261979.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
C:\Users\julie\AppData\Local\Temp\ipykernel_16564\1893261979.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [10]:
df_topic_sents_keywords.head(5)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,6,0.3025,"service, satisfied, price, good, customer, pri...","Best insurance, price, solutions, listening, s..."
1,18,0.1557,"insurance, years, without, pay, nothing, take,...","I am generally satisfied, except that you have..."
2,18,0.2269,"insurance, years, without, pay, nothing, take,...",Very affordable price Several options are avai...
3,6,0.1782,"service, satisfied, price, good, customer, pri...","I satisfy the service, a very fast response fr..."
4,1,0.2093,"contract, time, year, since, even, make, one, ...","Customer for more than 25 years, very disappoi..."


### K means

In [20]:
# Do a k-means clustering
kmeans = KMeans(n_clusters=5, random_state=0).fit(train_data['embedding'].to_list())
train_data['kmeans_cluster'] = kmeans.labels_

### DB SCAN

In [21]:
# Do a DBSCAN on the embeddings
clustering = DBSCAN(eps=0.1, min_samples=5).fit(train_data['embedding'].tolist())
train_data['dbscan_cluster'] = clustering.labels_

In [31]:
print("Number of clusters:", len(set(clustering.labels_)))
print("Number of reviews in the largest cluster:", max([len(train_data[train_data['dbscan_cluster'] == i]) for i in set(clustering.labels_)]))
print("Number of reviews in the smallest cluster:", min([len(train_data[train_data['dbscan_cluster'] == i]) for i in set(clustering.labels_)]))
print("Number of noise points:", len(train_data[train_data['dbscan_cluster'] == -1]))
print("Average number of reviews per cluster:", len(train_data) / len(set(clustering.labels_)))

Number of clusters: 4
Number of reviews in the largest cluster: 23491
Number of reviews in the smallest cluster: 5
Number of noise points: 23491
Average number of reviews per cluster: 6000.0


### PCA

In [ ]:
embedding_list = train_data['embedding'].to_list()
embedding_list

In [28]:
# Do a PCA to reduce the dimensionality of the embeddings
pca = PCA(n_components=3)
pca.fit(embedding_list)
embedding_pca = pca.transform(embedding_list)

# Filter out the noise points
df_filtered = train_data[train_data['dbscan_cluster'] != -1]
pca_filtered = PCA(n_components=3)
pca_filtered.fit(df_filtered['embedding'].tolist())
embeddings_pca_filtered = pca.transform(df_filtered['embedding'].tolist())

In [29]:
# Plot the PCA
from mpl_toolkits import mplot3d
import plotly.express as px

fig = px.scatter_3d(x=embedding_pca[:,0], y=embedding_pca[:,1], z=embedding_pca[:,2], color=train_data['kmeans_cluster'])
fig.show()

In [30]:
# Filter out the noise points
fig = px.scatter_3d(x=embeddings_pca_filtered[:,0], y=embeddings_pca_filtered[:,1], z=embeddings_pca_filtered[:,2], 
                    color=df_filtered['dbscan_cluster'])
fig.show()